In [ ]:
import itertools as it
import operator
import re
from collections import Counter
from pathlib import Path

import duckdb
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from pyarrow import csv
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.io as sio

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

# Config

In [ ]:
data_dir = Path(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/"
)

In [ ]:
arrow_ds = ds.dataset(list(data_dir.glob("*.arrow")), format="arrow")
arrow_ds2 = ds.dataset(list(data_dir.glob("*.arrow"))[:1], format="arrow")
parquet_ds = ds.dataset(list(data_dir.glob("*.parquet")), format="parquet")

# Group by path

In [ ]:
%%time
res = duckdb.execute("SELECT path, list(name) FROM arrow_ds GROUP BY path")

In [ ]:
2**32 / 1e6

In [ ]:
x = res.fetch_record_batch(10)

In [ ]:
x.read_next_batch()

In [ ]:
df = pl.scan_pyarrow_dataset(arrow_ds2)

In [ ]:
df.group_by("path").agg(pl.col("name"), pl.count()).filter(
    pl.col("count") > 5
).collect()

# Quality

In [ ]:
%%time
qs_duplex = duckdb.sql("SELECT qs FROM arrow_ds WHERE contains(name, ';')").arrow()
qs_simplex = duckdb.sql("SELECT qs FROM arrow_ds WHERE NOT contains(name, ';')").arrow()

In [ ]:
plt.hist(qs_duplex, bins=100, density=True)
plt.hist(qs_simplex, bins=100, density=True);

# Quality of true vs. false pairs

In [ ]:
reads = pa.ipc.open_file(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/channel-1_merged.arrow"
).read_all()

In [ ]:
name_col = reads.column("name")
path_col = reads.column("path")
qs_col = reads.column("qs")

In [ ]:
name_to_barcode = {}
name_to_path = {}
name_to_qs = {}
for idx in trange(len(reads)):
    name = name_col[idx].as_py()
    path = path_col[idx].as_py()
    if path is None:
        continue  # skip unaligned reads
    path = set([s[1:] for s in path])
    name_to_path[name] = path
    if ("BC:BIT0=0" in path or "BC:BIT0=1" in path) and (
        "BC:BIT29=0" in path or "BC:BIT29=1" in path
    ):
        barcode = tuple(f"BC:BIT{bit}=1" in path for bit in range(30))
        name_to_barcode[name] = barcode
        name_to_qs[name] = qs_col[idx].as_py()
duplex_ids = set(name_col.filter(pc.match_substring(name_col, ";")).to_pylist())
num_mapped_reads = 0
num_barcodes_match = 0
qs_match = []
qs_mismatch = []
for duplex_id in tqdm(duplex_ids):
    parent_ids = duplex_id.split(";")
    if (
        duplex_id in name_to_barcode
        and parent_ids[0] in name_to_barcode
        and parent_ids[1] in name_to_barcode
    ):
        num_mapped_reads += 1
        if (
            name_to_barcode[duplex_id]
            == name_to_barcode[parent_ids[0]]
            == name_to_barcode[parent_ids[1]]
        ):
            qs_match.append(name_to_qs[duplex_id])
            num_barcodes_match += 1
        else:
            qs_mismatch.append(name_to_qs[duplex_id])
# experiment_name = gaf_filename.parts[-2][: gaf_filename.parts[-2].index("_") + 2]
# num_total_reads = len(reads)
# num_duplex_reads = len(duplex_ids)
# rows.append(
#     {
#         "Experiment": experiment_name,
#         "Total Reads": num_total_reads,
#         "Duplex Reads": num_duplex_reads,
#         "Mapped Reads": num_mapped_reads,
#         "Barcodes Match": num_barcodes_match,
#         "Duplex Rate": num_duplex_reads / num_total_reads,
#         "Pair Mismatch Rate": 1 - num_barcodes_match / num_mapped_reads,
#     }
# )

In [ ]:
plt.hist(qs_match, bins=100)
plt.hist(qs_mismatch, bins=100);

# Length distributions

In [ ]:
plt.hist(
    pc.utf8_length(reads.column("read_seq"))
    .filter(pc.invert(pc.match_substring(name_col, ";")))
    .to_numpy(),
    bins=100,
    log=True,
)
plt.hist(
    pc.utf8_length(reads.column("read_seq"))
    .filter(pc.match_substring(name_col, ";"))
    .to_numpy(),
    bins=100,
    log=True,
);

In [ ]:
plt.hist(
    pc.utf8_length(reads.column("read_seq"))
    .filter(pc.invert(pc.match_substring(name_col, ";")))
    .to_numpy(),
    bins=100,
    log=False,
    cumulative=True,
    density=True,
)
plt.hist(
    pc.utf8_length(reads.column("read_seq"))
    .filter(pc.match_substring(name_col, ";"))
    .to_numpy(),
    bins=100,
    log=False,
    cumulative=True,
    density=True,
);

In [ ]:
bam_filename = "/home/jqs1/scratch/jqs1/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/output/dorado_0.4.0/bam/channel-1_merged.bam"

In [ ]:
bam = pysam.AlignmentFile(bam_filename, check_sq=False)

In [ ]:
tags = {}

In [ ]:
bam.reset()
for read in tqdm(bam.fetch(until_eof=True)):
    tags.update(dict(read.tags))

In [ ]:
tag

In [ ]:
read.get_tag("pi")

In [ ]:
tags

# Scratch

In [ ]:
%%time
duckdb.sql("SELECT count(distinct(name)) FROM arrow_ds WHERE path IS NOT NULL;").arrow()

In [ ]:
table = duckdb.read_parquet(str(data_dir / "*.parquet"))

In [ ]:
duckdb.execute(
    f"CREATE VIEW reads AS SELECT * FROM read_parquet('{data_dir / '*.parquet'}');"
)

In [ ]:
%%time
duckdb.sql("SELECT count(distinct(name)) FROM reads WHERE path IS NOT NULL;").arrow()

In [ ]:
duckdb.sql("SELECT COUNT(name) FROM READS;")

In [ ]:
lengths = duckdb.sql(
    "SELECT LENGTH(read_seq) FROM READS WHERE path NOT NULL;"
).fetchnumpy()

In [ ]:
plt.hist(lengths["length(read_seq)"], bins=100, log=True);

In [ ]:
plt.hist(
    lengths["length(read_seq)"][lengths["length(read_seq)"] < 20000], bins=100, log=True
);

In [ ]:
read_groups = duckdb.sql(
    "SELECT path, LIST(name), COUNT(name) FROM READS GROUP BY path;"
).arrow()

In [ ]:
read_groups  # [0][1000]

In [ ]:
plt.hist(read_groups.column('count("name")'), bins=100, log=True, range=(0, 250));

In [ ]:
read_groups.nbytes / 1e6

# False pairs vs. partial barcodes

In [ ]:
%%time
duplex_ids = set(k for k in reads.keys() if ";" in k)
simplex_ids = set(k for k in reads.keys() if ";" not in k)

In [ ]:
%%time
total_reads = 0
complete_barcodes = 0
name_to_barcode = {}
name_to_path = {}
for table in tqdm(sio.iter_gaf(gaf_filename)):
    name_col = table.column("name")
    path_col = table.column("path")
    for idx in range(len(table)):
        name = name_col[idx].as_py()
        path = set([s[1:] for s in path_col[idx].as_py()])
        name_to_path[name] = path
        total_reads += 1
        if ("BC:BIT0=0" in path or "BC:BIT0=1" in path) and (
            "BC:BIT29=0" in path or "BC:BIT29=1" in path
        ):
            complete_barcodes += 1
            barcode = tuple(f"BC:BIT{bit}=1" in path for bit in range(30))
            name_to_barcode[name] = barcode

In [ ]:
def partial_barcode_mismatches(a, b):
    return set(k for k, v in Counter(s.split("=")[0] for s in (a ^ b)).items() if v > 1)

In [ ]:
def mapping_status(id_, name_to_barcode, name_to_path):
    if id_ in name_to_barcode:
        return 0
    elif id_ in name_to_path:
        return 1
    else:
        return 2


STATUS_TO_NAME = {0: "Complete", 1: "Partial", 2: "Unmapped"}

census = pd.DataFrame(
    np.zeros((3, 6), dtype=np.uint32),
    columns=[
        "Complete/Complete",
        "Complete/Partial",
        "Complete/Unmapped",
        "Partial/Partial",
        "Partial/Unmapped",
        "Unmapped/Unmapped",
    ],
    index=["Duplex complete", "Duplex partial", "Duplex unmapped"],
)
total_reads = census.copy()
matches = census.copy()
mismatches = census.copy()

for duplex_id in tqdm(duplex_ids):
    parent_ids = duplex_id.split(";")
    parent_statuses = sorted(
        [
            mapping_status(parent_ids[0], name_to_barcode, name_to_path),
            mapping_status(parent_ids[1], name_to_barcode, name_to_path),
        ]
    )
    parent_col = "/".join(STATUS_TO_NAME[i] for i in parent_statuses)
    if duplex_id in name_to_barcode:
        row = "Duplex complete"
    elif duplex_id in name_to_path:
        row = "Duplex partial"
    else:
        row = "Duplex unmapped"
    total_reads.loc[row, parent_col] += 1
    if parent_statuses[0] != 2 and parent_statuses[1] != 2:
        if partial_barcode_mismatches(
            name_to_path[parent_ids[0]], name_to_path[parent_ids[1]]
        ):
            # if name_to_barcode[parent_ids[0]] == name_to_barcode[parent_ids[1]]:
            #    print(">>>",duplex_id);0/0
            # print(row, parent_col, partial_barcode_mismatches(name_to_path[parent_ids[0]], name_to_path[parent_ids[1]]));0/0
            mismatches.loc[row, parent_col] += 1
        else:
            matches.loc[row, parent_col] += 1

In [ ]:
total_reads

In [ ]:
matches

In [ ]:
mismatches

# False pairs summary

In [ ]:
data_dir = Path("/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/")

reads_filenames = [
    data_dir / "230707_repressilators/dorado_0.4.0/channel-135_merged.arrow",
    data_dir / "230726_carlos/dorado_0.4.0/channel-100_merged.arrow",
    data_dir / "230818_bcd_rbses/dorado_0.4.0/channel-100_merged.arrow",
    data_dir / "230818_repressilators/dorado_0.4.0/channel-1032_merged.arrow",
    data_dir / "230922_bcd_rbses_constitutive/dorado_0.4.0/channel-100_merged.arrow",
]

rows = []

for reads_filename in tqdm(reads_filenames):
    reads = pa.ipc.open_file(reads_filename)
    name_to_barcode = {}
    name_to_path = {}
    duplex_ids = set()
    num_total_reads = 0
    for batch_idx in trange(reads.num_record_batches):
        batch = reads.get_record_batch(batch_idx)
        name_col = batch.column("name")
        path_col = batch.column("path")
        duplex_ids.update(
            name_col.filter(pc.match_substring(name_col, ";")).to_pylist()
        )
        num_total_reads += len(batch)
        for idx in range(len(batch)):
            name = name_col[idx].as_py()
            path = path_col[idx].as_py()
            if path is None:
                continue  # skip unaligned reads
            path = set([s[1:] for s in path])
            name_to_path[name] = path
            if ("BC:BIT0=0" in path or "BC:BIT0=1" in path) and (
                "BC:BIT29=0" in path or "BC:BIT29=1" in path
            ):
                barcode = tuple(f"BC:BIT{bit}=1" in path for bit in range(30))
                name_to_barcode[name] = barcode
    num_mapped_reads = 0
    num_barcodes_match = 0
    for duplex_id in duplex_ids:
        parent_ids = duplex_id.split(";")
        if (
            duplex_id in name_to_barcode
            and parent_ids[0] in name_to_barcode
            and parent_ids[1] in name_to_barcode
        ):
            num_mapped_reads += 1
            if (
                name_to_barcode[duplex_id]
                == name_to_barcode[parent_ids[0]]
                == name_to_barcode[parent_ids[1]]
            ):
                num_barcodes_match += 1
    experiment_name = reads_filename.parts[-3][
        : reads_filename.parts[-3].index("_") + 2
    ]
    num_duplex_reads = len(duplex_ids)
    rows.append(
        {
            "Experiment": experiment_name,
            "Total Reads": num_total_reads,
            "Duplex Reads": num_duplex_reads,
            "Mapped Reads": num_mapped_reads,
            "Barcodes Match": num_barcodes_match,
            "Duplex Rate": num_duplex_reads / num_total_reads,
            "Pair Mismatch Rate": 1 - num_barcodes_match / num_mapped_reads,
        }
    )

In [ ]:
pd.DataFrame(rows).style.format({"Pair Mismatch Rate": "{:.0%}"})

In [ ]:
print(pd.DataFrame(rows).to_markdown(tablefmt="github", index=False))